# Deblur with InceptionResNet-v2

In [4]:
#@title Installing and downloading model
!git clone https://github.com/styler00dollar/Colab-DeblurGANv2
!pip install fire
!pip install pretrainedmodels
!pip install gdown
%cd Colab-DeblurGANv2/models
!gdown --id 1UXcsRVW-6KF23_TNzxw-xC0SzaMfXOaR

Cloning into 'Colab-DeblurGANv2'...


c:\Users\India\Documents\Colab-DeblurGANv2\models


C:\Users\India\AppData\Roaming\Python\Python311\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]
C:\Users\India\AppData\Local\Programs\Python\Python311\Lib\site-packages\gdown\__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1UXcsRVW-6KF23_TNzxw-xC0SzaMfXOaR
From (redirected): https://drive.google.com/uc?id=1UXcsRVW-6KF23_TNzxw-xC0SzaMfXOaR&confirm=t&uuid=bde97d88-6d64-40b2-baf6-fdf63a6fb140
To: c:\Users\India\Documents\Colab-DeblurGANv2\models\fpn_inception.h5

  0%|          | 0.00/244M [00:00<?, ?B/s]
  0%|          | 524k/244M [00:00<01:25, 2.87MB/s]
  1%|          | 1.57M/244M [00:00<00:46, 5.22MB/s]
  1%|          | 2.6

In [2]:
%cd Colab-DeblurGANv2/

c:\Users\India\Documents\Colab-DeblurGANv2


C:\Users\India\AppData\Roaming\Python\Python311\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
!dir

 Volume in drive C has no label.
 Volume Serial Number is D651-8414

 Directory of c:\Users\India\Documents\Colab-DeblurGANv2

25-04-2024  22:18    <DIR>          .
25-04-2024  22:18    <DIR>          ..
25-04-2024  15:19                12 .gitignore
25-04-2024  15:19             2,776 adversarial_trainer.py
25-04-2024  15:19             3,508 aug.py
25-04-2024  15:19            32,894 Colab-DeblurGANv2.ipynb
25-04-2024  15:19    <DIR>          config
25-04-2024  15:19             5,320 dataset.py
25-04-2024  15:19    <DIR>          doc_images
25-04-2024  22:15    <DIR>          input_examples
25-04-2024  15:19             3,623 LICENSE
25-04-2024  15:19             2,186 metric_counter.py
25-04-2024  15:43    <DIR>          models
25-04-2024  20:14             5,519 predict.py
25-04-2024  15:19             6,033 README.md
25-04-2024  15:19               190 requirements.txt
25-04-2024  15:19             1,989 schedulers.py
25-04-2024  22:18    <DIR>          submit
25-04-2024  15:19  

In [6]:
#@title Changing predict.py
%%writefile predict.py
import os
from glob import glob
from typing import Optional

import cv2
import numpy as np
import torch
import yaml
from fire import Fire
from tqdm import tqdm

from aug import get_normalize
from models.networks import get_generator


class Predictor:
    def __init__(self, weights_path: str, model_name: str = ''):
        with open('config/config.yaml') as cfg:
            config = yaml.load(cfg, Loader=yaml.FullLoader)
        model = get_generator(model_name or config['model'])
        model.load_state_dict(torch.load(weights_path)['model'])
        self.model = model.cuda()
        self.model.train(True)
        # GAN inference should be in train mode to use actual stats in norm layers,
        # it's not a bug
        self.normalize_fn = get_normalize()

    @staticmethod
    def _array_to_batch(x):
        x = np.transpose(x, (2, 0, 1))
        x = np.expand_dims(x, 0)
        return torch.from_numpy(x)

    def _preprocess(self, x: np.ndarray, mask: Optional[np.ndarray]):
        x, _ = self.normalize_fn(x, x)
        if mask is None:
            mask = np.ones_like(x, dtype=np.float32)
        else:
            mask = np.round(mask.astype('float32') / 255)

        h, w, _ = x.shape
        block_size = 32
        min_height = (h // block_size + 1) * block_size
        min_width = (w // block_size + 1) * block_size

        pad_params = {'mode': 'constant',
                      'constant_values': 0,
                      'pad_width': ((0, min_height - h), (0, min_width - w), (0, 0))
                      }
        x = np.pad(x, **pad_params)
        mask = np.pad(mask, **pad_params)

        return map(self._array_to_batch, (x, mask)), h, w

    @staticmethod
    def _postprocess(x: torch.Tensor) -> np.ndarray:
        x, = x
        x = x.detach().cpu().float().numpy()
        x = (np.transpose(x, (1, 2, 0)) + 1) / 2.0 * 255.0
        return x.astype('uint8')

    def __call__(self, img: np.ndarray, mask: Optional[np.ndarray], ignore_mask=True) -> np.ndarray:
        (img, mask), h, w = self._preprocess(img, mask)
        with torch.no_grad():
            inputs = [img.cuda()]
            if not ignore_mask:
                inputs += [mask]
            pred = self.model(*inputs)
        return self._postprocess(pred)[:h, :w, :]

def process_video(pairs, predictor, output_dir):
    for video_filepath, mask in tqdm(pairs):
        video_filename = os.path.basename(video_filepath)
        output_filepath = os.path.join(output_dir, os.path.splitext(video_filename)[0]+'_deblur.mp4')
        video_in = cv2.VideoCapture(video_filepath)
        fps = video_in.get(cv2.CAP_PROP_FPS)
        width = int(video_in.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(video_in.get(cv2.CAP_PROP_FRAME_HEIGHT))
        total_frame_num = int(video_in.get(cv2.CAP_PROP_FRAME_COUNT))
        video_out = cv2.VideoWriter(output_filepath, cv2.VideoWriter_fourcc(*'MP4V'), fps, (width, height))
        tqdm.write(f'process {video_filepath} to {output_filepath}, {fps}fps, resolution: {width}x{height}')
        for frame_num in tqdm(range(total_frame_num), desc=video_filename):
            res, img = video_in.read()
            if not res:
                break
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            pred = predictor(img, mask)
            pred = cv2.cvtColor(pred, cv2.COLOR_RGB2BGR)
            video_out.write(pred)

def main(img_pattern: str,
         mask_pattern: Optional[str] = None,
         weights_path='/content/Colab-DeblurGANv2/models/fpn_inception.h5',
         out_dir='submit/',
         side_by_side: bool = False,
         video: bool = False):
    def sorted_glob(pattern):
        return sorted(glob(pattern))

    imgs = sorted_glob(img_pattern)
    masks = sorted_glob(mask_pattern) if mask_pattern is not None else [None for _ in imgs]
    pairs = zip(imgs, masks)
    names = sorted([os.path.basename(x) for x in glob(img_pattern)])
    predictor = Predictor(weights_path=weights_path)

    os.makedirs(out_dir, exist_ok=True)
    if not video:
        for name, pair in tqdm(zip(names, pairs), total=len(names)):
            f_img, f_mask = pair
            img, mask = map(cv2.imread, (f_img, f_mask))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            pred = predictor(img, mask)
            if side_by_side:
                pred = np.hstack((img, pred))
            pred = cv2.cvtColor(pred, cv2.COLOR_RGB2BGR)
            cv2.imwrite(os.path.join(out_dir, name),
                        pred)
    else:
        process_video(pairs, predictor, out_dir)


if __name__ == '__main__':
    Fire(main)

UsageError: Line magic function `%%writefile` not found.


In [8]:
#@title Downloading model from my Google Drive (Very fast download)
!mkdir /root/
!mkdir /root/.cache/
!mkdir /root/.cache/torch/
!mkdir /root/.cache/torch/checkpoints/
%cd /root/.cache/torch/checkpoints/
!gdown --id 1y6GeaoWjhqjRjrXuZvCYEQYlblZGkE6X

The syntax of the command is incorrect.
The syntax of the command is incorrect.
The syntax of the command is incorrect.


[WinError 3] The system cannot find the path specified: '/root/.cache/torch/checkpoints/'
c:\Users\India\Documents\Colab-DeblurGANv2\models


The syntax of the command is incorrect.
C:\Users\India\AppData\Roaming\Python\Python311\site-packages\IPython\core\magics\osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
C:\Users\India\AppData\Local\Programs\Python\Python311\Lib\site-packages\gdown\__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1y6GeaoWjhqjRjrXuZvCYEQYlblZGkE6X
From (redirected): https://drive.google.com/uc?id=1y6GeaoWjhqjRjrXuZvCYEQYlblZGkE6X&confirm=t&uuid=9e9d79c0-9716-44bd-963b-38a594f60675
To: c:\Users\India\Documents\Colab-DeblurGANv2\models\inceptionresnetv2-520b38e4.pth

  0%|          | 0.00/224M [00:00<?, ?B/s]
  0%|          | 524k/224M [00:08<1:00:20, 61.7kB/s]
  0%|          | 1.05M/224

In [9]:
!dir

 Volume in drive C has no label.
 Volume Serial Number is D651-8414

 Directory of c:\Users\India\Documents\Colab-DeblurGANv2\models

25-04-2024  15:42    <DIR>          .
25-04-2024  15:42    <DIR>          ..
25-04-2024  15:19             5,125 fpn_densenet.py
25-04-2024  15:20       244,480,361 fpn_inception.h5
25-04-2024  15:19             6,633 fpn_inception.py
25-04-2024  15:19             5,974 fpn_inception_simple.py
25-04-2024  15:19             5,835 fpn_mobilenet.py
25-04-2024  15:19             5,799 fpn_mobilenet.py~HEAD
25-04-2024  15:19            10,836 losses.py
25-04-2024  15:19             4,340 mobilenet_v2.py
25-04-2024  15:19             1,117 models.py
25-04-2024  15:19            13,102 networks.py
25-04-2024  15:19            16,763 senet.py
25-04-2024  15:19             5,776 unet_seresnext.py
25-04-2024  15:19                 0 __init__.py
              13 File(s)    244,561,661 bytes
               2 Dir(s)  60,690,583,552 bytes free


In [22]:
%cd /Users/India/Documents/Colab-DeblurGANv2

c:\Users\India\Documents\Colab-DeblurGANv2


C:\Users\India\AppData\Roaming\Python\Python311\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [15]:
!python predict.py "9.jpeg"


  0%|          | 0/1 [00:00<?, ?it/s][ WARN:0@17.467] global loadsave.cpp:248 cv::findDecoder imread_(''): can't open/read file: check file path/integrity
c:\Users\India\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\functional.py:3809: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")

100%|██████████| 1/1 [00:13<00:00, 13.21s/it]
